In [ ]:
# Utility functions for objects
def get_size(grid):
    ''' Returns the size of the grid in 2 axis '''
    return (len(grid), len(grid[0]))

def get_middle(coords):
    ''' Returns the middle of the coordinates '''
    x_sum, y_sum = 0, 0
    for (x, y) in coords:
        x_sum += int(x)
        y_sum += int(y)
    return (int(x_sum / len(coords) + 0.5), int(y_sum / len(coords) + 0.5))

def get_anchor(coords):
    ''' Returns the top left, and bottom right anchor of the shape '''
    min_x, min_y = 10000, 10000
    max_x, max_y = 0, 0
    for (x, y) in coords:
        min_x = min(x, min_x)
        min_y = min(y, min_y)
        max_x = max(x, max_x)
        max_y = max(y, max_y)
    return (min_x, min_y), (max_x, max_y)

def get_pixel_coords(grid):
    ''' Gets the coordinates of all pixel values '''
    pixel_coord = {}
    for row in range(len(grid)):
        for col in range(len(grid[0])):
            value = grid[row][col]
            if value != 0:  # We assume '0' or '.' is empty space
                if value in pixel_coord:
                    pixel_coord[value].append((row, col))
                else:
                    pixel_coord[value] = [(row, col)]
    return dict(sorted(pixel_coord.items(), key=lambda x: -len(x[1])))

def obj_to_coords(obj):
    coords = []
    x, y = obj['tl']
    height, width = len(obj['grid']), len(obj['grid'][0])
    for i in range(height):
        for j in range(width):
            if obj['grid'][i][j] != 0:
                coords.append((x + i, y + j))
    return coords

def create_object(grid, coords):
    ''' Create an object based on the existing grid and the coordinates of it '''
    (min_x, min_y), (max_x, max_y) = get_anchor(coords)
    newgrid = [[0 for _ in range(max_y - min_y + 1)] for _ in range(max_x - min_x + 1)]
    for (x, y) in coords:
        if grid[x][y] == 0:
            newgrid[x - min_x][y - min_y] = -1  # Indicate empty spot with -1
        else:
            newgrid[x - min_x][y - min_y] = grid[x][y]
    return {'tl': (min_x, min_y), 'grid': newgrid}

# Test case
assert get_pixel_coords([[1, 1], [4, 6]]) == {1: [(0, 0), (0, 1)], 4: [(1, 0)], 6: [(1, 1)]}

def get_objects(grid, diag=False, multicolor=False, by_row=False, by_col=False, by_color=False, more_info=True):
    rows = len(grid)
    cols = len(grid[0])
    visited = set()
    objects = []
    missing_color = False

    # Check whether there is a missing color (using '10' to denote missing color)
    for each in grid:
        for cell in each:
            if cell == 10:
                missing_color = True

    def is_valid(grid, row, col, value):
        # Multicolor can return any cell as long as it is not visited and not a blank
        if multicolor:
            return 0 <= row < rows and 0 <= col < cols and (row, col) not in visited and grid[row][col] != 0
        else:  
            return 0 <= row < rows and 0 <= col < cols and (row, col) not in visited and grid[row][col] == value

    def dfs(grid, row, col, value):
        if is_valid(grid, row, col, value):
            visited.add((row, col))
            object_coords.add((row, col))

            if not by_row:
                dfs(grid, row - 1, col, value)  # up
                dfs(grid, row + 1, col, value)  # down
            if not by_col:
                dfs(grid, row, col - 1, value)  # left
                dfs(grid, row, col + 1, value)  # right
            if not by_row and not by_col and diag:
                dfs(grid, row - 1, col - 1, value)  # top-left diagonal
                dfs(grid, row - 1, col + 1, value)  # top-right diagonal
                dfs(grid, row + 1, col - 1, value)  # bottom-left diagonal
                dfs(grid, row + 1, col + 1, value)  # bottom-right diagonal

    # If by_color, we don't need to do DFS
    if by_color:
        pixels = get_pixel_coords(grid)
        for key, value in pixels.items():
            object_coords = value
            object_dict = create_object(grid, object_coords)
            if more_info:
                object_dict['size'] = (len(object_dict['grid']), len(object_dict['grid'][0]))
                object_dict['cell_count'] = len(object_coords)
                object_dict['shape'] = [[1 if cell != 0 else 0 for cell in row] for row in object_dict['grid']]
            objects.append(object_dict)
            
    else:
        for row in range(rows):
            for col in range(cols):
                value = grid[row][col]
                if (row, col) not in visited:
                    if value == 0: continue
                    object_coords = set()
                    dfs(grid, row, col, value)
                    object_dict = create_object(grid, object_coords)
                    if more_info:
                        object_dict['size'] = (len(object_dict['grid']), len(object_dict['grid'][0]))
                        object_dict['cell_count'] = len(object_coords)
                        object_dict['shape'] = [[1 if cell != 0 else 0 for cell in row] for row in object_dict['grid']]
                    objects.append(object_dict)

        # If there's no color '10', perform additional processing for inner objects
        if not missing_color:
            multicolor = False
            new_objects = []
            for obj in objects:
                visited = set()
                newgrid = obj['grid']
                rows = len(newgrid)
                cols = len(newgrid[0])
                for row in range(rows):
                    for col in range(cols):
                        if (row, col) not in visited:
                            if newgrid[row][col] == 0:
                                object_coords = set()
                                dfs(newgrid, row, col, 0)
                                boundary = False
                                for x, y in object_coords:
                                    if x == 0 or x == len(newgrid) - 1 or y == 0 or y == len(newgrid[0]) - 1:
                                        boundary = True
                                if boundary: continue
                                object_dict = create_object(newgrid, object_coords)
                                cur_x, cur_y = object_dict['tl']
                                base_x, base_y = obj['tl']
                                object_dict['tl'] = (cur_x + base_x, cur_y + base_y)
                                if more_info:
                                    object_dict['size'] = (len(object_dict['grid']), len(object_dict['grid'][0]))
                                    object_dict['cell_count'] = len(object_coords)
                                    object_dict['shape'] = [[1 if cell != 0 else 0 for cell in row] for row in object_dict['grid']]
                                new_objects.append(object_dict)
            objects.extend(new_objects)
    return objects

def combine_object(obj1, obj2):
    # if not an instance of object, create it
    if not isinstance(obj1, dict): obj1 = {'tl': (0, 0), 'grid': obj1}
    if not isinstance(obj2, dict): obj2 = {'tl': (0, 0), 'grid': obj2}
    grid = empty_grid(30, 30)
    grid = fill_grid(grid, obj1['tl'], obj1['grid'])
    grid = fill_grid(grid, obj2['tl'], obj2['grid'])
    obj_coords = obj_to_coords(obj1)
    obj_coords2 = obj_to_coords(obj2)
    obj_coords.extend(obj_coords2)
    return create_object(grid, obj_coords)

def tight_fit(grid):
    objects = get_objects(grid)
    obj = objects[0]
    for each in objects[1:]:
        obj = combine_object(obj, each)
    return obj['grid']

def fill_grid(grid, tl, pattern):
    x, y = tl
    if not isinstance(pattern, list): pattern = [[pattern]]
    for row in range(len(pattern)):
        for col in range(len(pattern[0])):
            if 0 <= row + x < len(grid) and 0 <= col + y < len(grid[0]): 
                if pattern[row][col] != -1:
                    grid[row + x][col + y] = pattern[row][col]
    return grid

def fill_object(grid, obj, align=False):
    if align: return obj['grid']
    return fill_grid(grid, obj['tl'], obj['grid'])

def empty_grid(row, col):
    return [[0 for _ in range(col)] for _ in range(row)]

def crop_grid(grid, tl, br): 
    return [[grid[i][j] for j in range(tl[1], br[1] + 1)] for i in range(tl[0], br[0] + 1)]

def fill_between_coords(grid, coord1, coord2, value): 
    # fill up a point
    if coord1 == coord2:
        grid[coord1[0]][coord1[1]] = value
        return grid
    
    # fill up a line
    row_diff = coord1[0] - coord2[0]
    col_diff = coord1[1] - coord2[1]
    maxdist = max(abs(row_diff), abs(col_diff))
    height, width = len(grid), len(grid[0])
    for i in range(maxdist + 1):
        row_pos, col_pos = coord1[0] - (i * row_diff) // maxdist, coord1[1] - (i * col_diff) // maxdist
        if 0 <= row_pos < height and 0 <= col_pos < width:
            grid[row_pos][col_pos] = value
    return grid

def rotate_clockwise(grid, degree=90):
    newgrid = copy.deepcopy(grid)
    for i in range(degree // 90):
        # Transpose the array
        transposed_grid = [[newgrid[j][i] for j in range(len(newgrid))] for i in range(len(newgrid[0]))]

        # Reverse each row of the transposed array
        newgrid = [row[::-1] for row in transposed_grid]
    return newgrid

def horizontal_flip(grid):
    return [row[::-1] for row in grid]

def vertical_flip(grid):
    return [row for row in grid[::-1]]

def fill_value(grid, pos, value):
    x, y = pos
    if x < 0 or x >= len(grid) or y < 0 or y >= len(grid[0]): return grid
    grid[x][y] = value
    return grid

def replace(grid, pattern1, pattern2):
    if not isinstance(pattern1, list): pattern1 = [[pattern1]]
    if not isinstance(pattern2, list): pattern2 = [[pattern2]]
    height, width = len(pattern1), len(pattern1[0])
    for i in range(len(grid) - height + 1):
        for j in range(len(grid[0]) - width + 1):
            if crop_grid(grid, (i, j), (i + height - 1, j + width - 1)) == pattern1:
                grid = fill_grid(grid, (i, j), pattern2)
    return grid

def fill_rect(grid, tl, br, value):
    for row in range(tl[0], br[0] + 1):
        for col in range(tl[1], br[1] + 1):
            grid = fill_value(grid, (row, col), value)
    return grid

def fill_row(grid, row_num, value, start_col=0, end_col=30):
    for col_num in range(start_col, end_col + 1):
        grid = fill_value(grid, (row_num, col_num), value)
    return grid

def fill_col(grid, col_num, value, start_row=0, end_row=30): 
    for row_num in range(start_row, end_row + 1):
        grid = fill_value(grid, (row_num, col_num), value)
    return grid

def change_object_pos(obj, new_tl):
    obj['tl'] = new_tl
    return obj

def change_object_color(obj, value):
    for row in range(len(obj['grid'])):
        for col in range(len(obj['grid'][0])):
            if obj['grid'][row][col] != 0:
                obj['grid'][row][col] = value
    return obj

def get_object_color(obj):
    for row in range(len(obj['grid'])):
        for col in range(len(obj['grid'][0])):
            if obj['grid'][row][col] != 0:
                return obj['grid'][row][col]
    return 0




In [ ]:
helper_functions = '''- get_objects(grid, diag=False, by_row=False, by_col=False, by_color=False, multicolor=False, more_info=True): 
Takes in grid, returns list of object dictionary: top-left coordinate of object ('tl'), 2D grid ('grid')
by_row views splits objects by grid rows, by_col splits objects by grid columns, by_color groups each color as one object, multicolor means object can be more than one color
Empty cells within objects are represented as -1
If more_info is True, also returns size of grid ('size'), cells in object ('cell_count'), shape of object ('shape')
- get_pixel_coords(grid): Returns a dictionary, with the keys the pixel values, values the list of coords, in sorted order from most number of pixels to least
- empty_grid(row, col): returns an empty grid of height row and width col
- crop_grid(grid, tl, br): returns cropped section from top left to bottom right of the grid
- tight_fit(grid): returns grid with all blank rows and columns removed
- combine_object(obj_1, obj_2): returns combined object from obj_1 and obj_2. if overlap, obj_2 overwrites obj_1
- rotate_clockwise(grid, degree=90): returns rotated grid clockwise by a degree of 90, 180, 270 degrees
- horizontal_flip(grid): returns a horizontal flip of the grid
- vertical_flip(grid): returns a vertical flip of the grid
- replace(grid, grid_1, grid_2): replaces all occurrences of grid_1 with grid_2 in grid
- get_object_color(obj): returns color of object. if multicolor, returns first color only
- change_object_color(obj, value): changes the object color to value
- fill_object(grid, obj, align=False): fills grid with object. If align is True, makes grid same size as object
- fill_row(grid, row_num, value, start_col=0, end_col=30): fills output grid with a row of value at row_num from start_col to end_col (inclusive)
- fill_col(grid, col_num, value, start_row=0, end_row=30): fills output grid with a column of value at col_num from start_row to end_row (inclusive)
- fill_between_coords(grid, coord_1, coord_2, value): fills line between coord_1 and coord_2 with value
- fill_rect(grid, tl, br, value): fills grid from tl to br with value. useful to create rows, columns, rectangles
- fill_value(grid, pos, value): fills grid at position with value
'''

# Assertions updated to use integer representations (0 for empty cells, -1 for placeholders)
assert get_objects([[1, 1, 1], [1, 0, 1], [1, 1, 1]], more_info=False) == [{'tl': (0, 0), 'grid': [[1, 1, 1], [1, 0, 1], [1, 1, 1]]}, {'tl': (1, 1), 'grid': [[-1]]}]
assert get_pixel_coords([[1, 1], [4, 6]]) == {1: [(0, 0), (0, 1)], 4: [(1, 0)], 6: [(1, 1)]}
assert empty_grid(3, 2) == [[0, 0], [0, 0], [0, 0]]
assert crop_grid([[1, 1, 2], [0, 1, 2]], (0, 0), (1, 1)) == [[1, 1], [0, 1]]
assert tight_fit([[0, 0, 0], [0, 1, 0], [0, 0, 0]]) == [[1]]
assert combine_object({'tl': (0, 0), 'grid': [[1, 1], [1, 0]]}, {'tl': (1, 1), 'grid': [[6]]}) == {'tl': (0, 0), 'grid': [[1, 1], [1, 6]]}
assert rotate_clockwise([[1, 2], [4, 5]], 90) == [[4, 1], [5, 2]]
assert rotate_clockwise([[1, 2], [4, 5]], 270) == [[2, 5], [1, 4]]
assert horizontal_flip([[1, 2, 3], [4, 5, 6]]) == [[3, 2, 1], [6, 5, 4]]
assert vertical_flip([[1, 2, 3], [4, 5, 6]]) == [[4, 5, 6], [1, 2, 3]]
assert replace([[1, 0], [1, 1]], [[1, 1]], [[3, 3]]) == [[1, 0], [3, 3]]
assert change_object_color({'tl': (0, 0), 'grid': [[1, 0]]}, 2) == {'tl': (0, 0), 'grid': [[2, 0]]}
assert get_object_color({'tl': (0, 0), 'grid': [[1, 0]]}) == 1
assert fill_object([[0, 0], [0, 0]], {'tl': (0, 1), 'grid': [[3], [3]]}) == [[0, 3], [0, 3]]
assert fill_value([[0, 1], [0, 1]], (1, 1), 2) == [[0, 1], [0, 2]]
assert fill_row([[1, 1], [3, 1]], 0, 2) == [[2, 2], [3, 1]]
assert fill_col([[1, 1], [3, 1]], 0, 2) == [[2, 1], [2, 1]]
assert fill_rect([[1, 1], [3, 1]], (0, 0), (1, 1), 2) == [[2, 2], [2, 2]]
assert fill_between_coords([[0, 0]], (0, 0), (0, 1), 1) == [[1, 1]]


## Conditional Functions
- Functions to help with deciding when to use helper functions

In [ ]:

def object_contains_color(obj, value):
    for row in range(len(obj['grid'])):
        for col in range(len(obj['grid'][0])):
            if obj['grid'][row][col] == value:
                return True
    return False

def on_same_line(coord1, coord2, line_type):
    '''Returns True/False if coord1 is on the same line as coord2. line_type can be one of ['row', 'col', 'diag']'''
    if line_type == 'row': return coord1[0] == coord2[0]
    if line_type == 'col': return coord1[1] == coord2[1]
    if line_type == 'diag': return coord1[0]-coord2[0] == coord1[1]-coord2[1]
    return False
conditional_functions = '''
object_contains_color(obj, value): returns True/False if object contains a certain value
on_same_line(coord_1, coord_2): Returns True/False if coord_1 is on the same line as coord_2. line_type can be one of ['row', 'col', 'diag']
'''

assert object_contains_color({'tl':(0,0),'grid':[['a']]},'a')==True
assert on_same_line((1,1),(1,2),'row')==True
assert on_same_line((1,1),(2,1),'col')==True
assert on_same_line((1,1),(2,2),'diag')==True